# Importing libraries

In [3]:
import csv
import os
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

csv.field_size_limit(1000000000)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fazal\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fazal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fazal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fazal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1000000000

# Reading the file and performing basic cleanup

In [4]:
import string

# read csv file from data folder
df = pd.read_csv(os.path.join('data', 'state-of-the-union.csv'), names=['year', 'speech'], skiprows=1)

df['speech'] = df['speech'].str.replace('\nState of the Union Address\n', '')
df['speech'] = df['speech'].str.replace('\nAddress to Joint Session of Congress \n', '')
df['speech'] = df['speech'].str.replace('\nAddress on Administration Goals (Budget Message)\n', '')
df['speech'] = df['speech'].str.replace('\nAddress on Administration Goals\n', '')
df['speech'] = df['speech'].str.replace('\nAddress to Congress \n', '')

df['president'] = df['speech']

df['president'] = df['president'].str.split('\n').str[0]
df['date'] = df['speech'].str.split('\n').str[1]

temp_date = df[df['date'] == 'Address on Administration Goals (Budget Message)']['speech'].str.split('\n').str[3]
df['date'][df['date'] == 'Address on Administration Goals (Budget Message)'] = temp_date.values[0]

# delete first 3 lines of speech
df['speech'] = df['speech'].str.split('\n').str[3:]
# make a string list
df['speech'] = df['speech'].str.join(' ')
# replace \ with ''
df['speech'] = df['speech'].str.replace('\\\'', '')
# decapitalize
df['speech'] = df['speech'].str.lower()
# remove punctuation
df['speech'] = df['speech'].str.replace('[{}]'.format(string.punctuation), '')

C:\Users\fazal\AppData\Local\Temp\ipykernel_5008\3938583509.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['speech'] = df['speech'].str.replace('\nAddress on Administration Goals (Budget Message)\n', '')
C:\Users\fazal\AppData\Local\Temp\ipykernel_5008\3938583509.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][df['date'] == 'Address on Administration Goals (Budget Message)'] = temp_date.values[0]
C:\Users\fazal\AppData\Local\Temp\ipykernel_5008\3938583509.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  df['speech'] = df['speech'].str.replace('\\\'', '')
C:\Users\fazal\AppData\Local\Temp\ipykernel_5008\3938583509.py:29: FutureWarning: The default value of regex will change fro

# Performing Lemmatization

In [3]:
# perform lemmatization
lemmatizer = WordNetLemmatizer()


def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in word_tokenize(text)]


df['speech'] = df['speech'].apply(lemmatize_text)
df['speech'] = df['speech'].apply(lambda x: [item for item in x if item not in stop_words])
df['speech'] = df['speech'].apply(lambda x: ' '.join(x))

# Performing Stemming

In [8]:
# # perform stemming
# stemmer = PorterStemmer()
#
#
# def stem_text(text):
#     return [stemmer.stem(w) for w in word_tokenize(text)]
#
#
# df['speech'] = df['speech'].apply(stem_text)
# df['speech'] = df['speech'].apply(lambda x: [item for item in x if item not in stop_words])
# df['speech'] = df['speech'].apply(lambda x: ' '.join(x))
#
# # remove punctuation
# # import string
# #
# # df['speech'] = df['speech'].str.replace('[{}]'.format(string.punctuation), '')


print(df['speech'].head(10))

0    fellowcitizen senat hou repr meet feel much sa...
1    fellowcitizen senat hou repr vain may expect p...
2    fellowcitizen senat hou repr abat satisfact me...
3    fellowcitizen senat hou repr sinc commenc term...
4    fellowcitizen senat hou repr call mind graciou...
5    fellowcitizen senat hou repr trust deceiv indu...
6    fellowcitizen senat hou repr recur intern situ...
7    gentlemen senat gentlemen hou repr wa time app...
8    gentlemen senat gentlemen hou repr rever resig...
9    gentlemen senat gentlemen hou repr peculiar sa...
Name: speech, dtype: object


In [6]:
# remove punctuation
import string

df['speech'] = df['speech'].str.replace('[{}]'.format(string.punctuation), '')
print(df['speech'].head(10))

0    fellowcitizen senat hous repres  meet feel muc...
1    fellowcitizen senat hous repres   vain may exp...
2    fellowcitizen senat hous repres  abat satisfac...
3    fellowcitizen senat hous repres  sinc commenc ...
4    fellowcitizen senat hous repres  call mind gra...
5    fellowcitizen senat hous repres  trust deceiv ...
6    fellowcitizen senat hous repres  recur intern ...
7    gentlemen senat gentlemen hous repres  wa time...
8    gentlemen senat gentlemen hous repres  rever r...
9    gentlemen senat gentlemen hous repres  peculia...
Name: speech, dtype: object


C:\Users\fazal\AppData\Local\Temp\ipykernel_18900\939221554.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['speech'] = df['speech'].str.replace('[{}]'.format(string.punctuation), '')


In [7]:
from gensim import corpora

# create a dictionary from a list of speeches
dictionary = corpora.Dictionary(df['speech'])

# convert the dictionary to a bag of words
corpus = [dictionary.doc2bow(speech) for speech in df['speech']]

print(corpus[0])

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [9]:
import gensim

tokens = gensim.utils.simple_preprocess("A value is trying to be set on a copy of a slice from a DataFrame See the caveats in the documentation", min_len=4)
print(tokens)

['value', 'trying', 'copy', 'slice', 'from', 'dataframe', 'caveats', 'documentation']


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords

# Download stopwords list
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# Interface lemma tokenizer from nltk with sklearn
# Thank you to kaggle user mjmurphy28 for this code
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`', '--', '-','...', 'american', 'america', 'world']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

# Lemmatize the stop words
tokenizer=LemmaTokenizer()
D = tokenizer(' '.join(stop_words))
documents = df['speech']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fazal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Lemmatize with POS tagging

In [7]:
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

#Return pos tag in wordnetlemmatizer format

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

#Create the lemmatoken
# Interface lemma tokenizer from nltk with sklearn
class POSLemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`', '--', '-', '...', 'american', 'america', 'world']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t, get_wordnet_pos(t)) for t in word_tokenize(doc) if t not in self.ignore_tokens]

# Lemmatize the stop words
pos_tokenizer=POSLemmaTokenizer()
pos_token_stop=pos_tokenizer(' '.join(stop_words))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fazal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


#  Creating Tfidf on no-POS tagged documents

In [8]:
# Create TF-idf model
vectorizer = TfidfVectorizer(stop_words=token_stop,
                              tokenizer=tokenizer)
#Fit transform current document
tfidf_doc = vectorizer.fit_transform(documents)

tfidf_array = tfidf_doc.toarray()
tfidf_df = pd.DataFrame(tfidf_array, columns = vectorizer.get_feature_names())
tfidf_df.info()

C:\Users\fazal\PycharmProjects\Aritificial Intelligence\Assignment 2\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Columns: 22564 entries, 000 to zooming
dtypes: float64(22564)
memory usage: 38.7 MB


# Creating TFIDF with POS tagged document

In [9]:
# Create TF-idf model
pos_vectorizer = TfidfVectorizer(stop_words=pos_token_stop,
                              tokenizer=pos_tokenizer)
#Fit transform current document
pos_tfidf_doc = pos_vectorizer.fit_transform(documents)

pos_tfidf_array = pos_tfidf_doc.toarray()
pos_tfidf_df = pd.DataFrame(pos_tfidf_array, columns = pos_vectorizer.get_feature_names())
pos_tfidf_df.info()

C:\Users\fazal\PycharmProjects\Aritificial Intelligence\Assignment 2\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Columns: 19696 entries, 000 to zoom
dtypes: float64(19696)
memory usage: 33.8 MB


# EDA on no-POS taggged docs

In [12]:
df['topic'] = np.arange(0,225)

#Print out the topic-representing words in each year SOTU speech:
for i in range(0, 225):
    topc = []
    topic_words = tfidf_df.iloc[i, :].sort_values().tail(5).reset_index()
    for word in topic_words['index']:
        topc.append(word)
    df['topic'][i] = topc

print(df['topic'].head(10))

C:\Users\fazal\AppData\Local\Temp\ipykernel_5008\4183290690.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topic'][i] = topc


0          [requisite, militia, object, u, attachment]
1           [object, may, subscription, united, state]
2           [chickamauga, tribe, law, upon, provision]
3              [shall, relate, warmest, united, state]
4    [inspector, insurrection, state, pennsylvania,...
5    [country, situation, tranquillity, gentleman, ...
6              [may, war, united, commissioner, state]
7     [gentleman, treaty, commissioner, united, state]
8      [state, gentleman, france, croix, commissioner]
9    [punctuality, united, state, philadelphia, gen...
Name: topic, dtype: object


# EDA on POS-tagged docs

In [13]:
df['pos_topic'] = np.arange(0,225)

#Print out the topic-representing words in each year SOTU speech:
for i in range(0, 225):
    topcs = []
    topic_words = pos_tfidf_df.iloc[i, :].sort_values().tail(5).reset_index()
    for word in topic_words['index']:
        topcs.append(word)
    df['pos_topic'][i] = topcs

print(df['pos_topic'].head(10))

C:\Users\fazal\AppData\Local\Temp\ipykernel_5008\728159892.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos_topic'][i] = topcs


0               [militia, object, u, make, attachment]
1           [object, may, subscription, united, state]
2           [chickamauga, tribe, law, upon, provision]
3             [militia, warlike, shall, united, state]
4    [inspector, insurrection, state, pennsylvania,...
5    [country, situation, tranquillity, gentleman, ...
6              [may, war, united, commissioner, state]
7     [gentleman, treaty, commissioner, united, state]
8      [state, gentleman, france, croix, commissioner]
9    [punctuality, united, state, philadelphia, gen...
Name: pos_topic, dtype: object


In [22]:
for i in range(0, len(df['year'].values)):
    print(i+1790, df['year'][i])

set1 = set(df['year'].values)
print(list(set1))

1790 1790
1791 1791
1792 1792
1793 1793
1794 1794
1795 1795
1796 1796
1797 1797
1798 1798
1799 1799
1800 1800
1801 1801
1802 1802
1803 1803
1804 1804
1805 1805
1806 1806
1807 1807
1808 1808
1809 1809
1810 1810
1811 1811
1812 1812
1813 1813
1814 1814
1815 1815
1816 1816
1817 1817
1818 1818
1819 1819
1820 1820
1821 1821
1822 1822
1823 1823
1824 1824
1825 1825
1826 1826
1827 1827
1828 1828
1829 1829
1830 1830
1831 1831
1832 1834
1833 1832
1834 1834
1835 1835
1836 1836
1837 1837
1838 1838
1839 1839
1840 1840
1841 1841
1842 1842
1843 1843
1844 1844
1845 1845
1846 1846
1847 1847
1848 1848
1849 1849
1850 1850
1851 1851
1852 1852
1853 1853
1854 1854
1855 1855
1856 1856
1857 1857
1858 1858
1859 1859
1860 1860
1861 1861
1862 1862
1863 1863
1864 1864
1865 1865
1866 1866
1867 1867
1868 1868
1869 1869
1870 1870
1871 1871
1872 1872
1873 1873
1874 1874
1875 1875
1876 1876
1877 1877
1878 1878
1879 1879
1880 1880
1881 1881
1882 1882
1883 1883
1884 1884
1885 1885
1886 1886
1887 1887
1888 1888
1889 1889
